<a href="https://colab.research.google.com/github/Robin-01/California-Coastal-Sea-Currents/blob/main/download_wind_data_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!echo "machine urs.earthdata.nasa.gov login alexmw password Washington12#$" > ~/.netrc
!chmod 0600 ~/.netrc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import requests
from datetime import datetime, timedelta

# Directory to save files
save_dir = "/content/drive/MyDrive/UW_DSO_Project/Wind Data"
os.makedirs(save_dir, exist_ok=True)

# Date range
start_date = datetime(2025, 1, 1)
end_date   = datetime(2025, 3, 31)  # change as needed

# Loop over all dates
date = start_date
while date <= end_date:
    # Construct filename and URL
    filename = f"CCMP_Wind_Analysis_{date.strftime('%Y%m%d')}_V03.1_L4.nc"
    file_path = os.path.join(save_dir, filename)
    url = f"https://data.remss.com/ccmp/v03.1/Y{date.year}/M{date.month:02d}/{filename}"

    # Check if file already exists
    if os.path.exists(file_path):
        print(f"✅ Already have {filename}")
    else:
        print(f"⬇️ Downloading {filename} ...")
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(file_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"   ✔ Saved {filename}")
        else:
            print(f"   ❌ Failed to download {filename} (status {response.status_code})")

    # Next day
    date += timedelta(days=1)


⬇️ Downloading CCMP_Wind_Analysis_20250101_V03.1_L4.nc ...
   ✔ Saved CCMP_Wind_Analysis_20250101_V03.1_L4.nc
⬇️ Downloading CCMP_Wind_Analysis_20250102_V03.1_L4.nc ...
   ✔ Saved CCMP_Wind_Analysis_20250102_V03.1_L4.nc
⬇️ Downloading CCMP_Wind_Analysis_20250103_V03.1_L4.nc ...
   ✔ Saved CCMP_Wind_Analysis_20250103_V03.1_L4.nc
⬇️ Downloading CCMP_Wind_Analysis_20250104_V03.1_L4.nc ...
   ✔ Saved CCMP_Wind_Analysis_20250104_V03.1_L4.nc
⬇️ Downloading CCMP_Wind_Analysis_20250105_V03.1_L4.nc ...
   ✔ Saved CCMP_Wind_Analysis_20250105_V03.1_L4.nc
⬇️ Downloading CCMP_Wind_Analysis_20250106_V03.1_L4.nc ...
   ✔ Saved CCMP_Wind_Analysis_20250106_V03.1_L4.nc
⬇️ Downloading CCMP_Wind_Analysis_20250107_V03.1_L4.nc ...
   ✔ Saved CCMP_Wind_Analysis_20250107_V03.1_L4.nc
⬇️ Downloading CCMP_Wind_Analysis_20250108_V03.1_L4.nc ...
   ✔ Saved CCMP_Wind_Analysis_20250108_V03.1_L4.nc
⬇️ Downloading CCMP_Wind_Analysis_20250109_V03.1_L4.nc ...
   ✔ Saved CCMP_Wind_Analysis_20250109_V03.1_L4.nc
⬇️ Downloa

In [5]:
import xarray as xr
import os

# Folder where all files are saved
data_dir = '/content/drive/MyDrive/UW_DSO_Project/Wind Data'
# Get a sorted list of all NetCDF files
all_files = sorted([
    os.path.join(data_dir, f)
    for f in os.listdir(data_dir)
    if f.startswith("CCMP_Wind_Analysis_2025") and f.endswith(".nc")
])

print(f"Found {len(all_files)} files for 2025")

# Open all files as one Dataset using xarray and Dask for lazy loading
ds_all = xr.open_mfdataset(
    all_files,
    combine="by_coords",  # combine along coordinates like time
    chunks={"time": 10}   # optional: chunking for large datasets
)

# Inspect the combined dataset
print(ds_all)


Found 90 files for 2025
<xarray.Dataset> Size: 6GB
Dimensions:    (time: 360, latitude: 720, longitude: 1440)
Coordinates:
  * latitude   (latitude) float32 3kB -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 6kB 0.125 0.375 0.625 ... 359.4 359.6 359.9
  * time       (time) datetime64[ns] 3kB 2025-01-01 ... 2025-03-31T18:00:00
Data variables:
    uwnd       (time, latitude, longitude) float32 1GB dask.array<chunksize=(4, 720, 1440), meta=np.ndarray>
    vwnd       (time, latitude, longitude) float32 1GB dask.array<chunksize=(4, 720, 1440), meta=np.ndarray>
    ws         (time, latitude, longitude) float32 1GB dask.array<chunksize=(4, 720, 1440), meta=np.ndarray>
    nobs       (time, latitude, longitude) float32 1GB dask.array<chunksize=(4, 720, 1440), meta=np.ndarray>
Attributes: (12/54)
    contact:                       Remote Sensing Systems, support@remss.com
    Conventions:                   CF-1.7 ACDD-1.3
    data_structure:                grid
 

In [6]:
!pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 71.9 MB/s eta 0:00:00


In [7]:
# California bounds
lon_min, lon_max = -131 % 360, -109 % 360  # convert -180..0 to 0..360
lat_min, lat_max = 24, 51

# Crop dataset
ds_ca = ds_all.sel(
    longitude=slice(lon_min, lon_max),
    latitude=slice(lat_min, lat_max)
)

print(ds_ca)


ds_ca
ds_day = ds_ca.sel(time="2025-03-18T12:00", method="nearest")
if ds_day['longitude'].size > 0:
    print(ds_day['uwnd'].min().values)
    print(ds_day['uwnd'].max().values)
    print(ds_day['uwnd'].mean().values)
else:
    print("⚠️ Selection returned empty array")



<xarray.Dataset> Size: 55MB
Dimensions:    (time: 360, latitude: 108, longitude: 88)
Coordinates:
  * latitude   (latitude) float32 432B 24.12 24.38 24.62 ... 50.38 50.62 50.88
  * longitude  (longitude) float32 352B 229.1 229.4 229.6 ... 250.4 250.6 250.9
  * time       (time) datetime64[ns] 3kB 2025-01-01 ... 2025-03-31T18:00:00
Data variables:
    uwnd       (time, latitude, longitude) float32 14MB dask.array<chunksize=(4, 108, 88), meta=np.ndarray>
    vwnd       (time, latitude, longitude) float32 14MB dask.array<chunksize=(4, 108, 88), meta=np.ndarray>
    ws         (time, latitude, longitude) float32 14MB dask.array<chunksize=(4, 108, 88), meta=np.ndarray>
    nobs       (time, latitude, longitude) float32 14MB dask.array<chunksize=(4, 108, 88), meta=np.ndarray>
Attributes: (12/54)
    contact:                       Remote Sensing Systems, support@remss.com
    Conventions:                   CF-1.7 ACDD-1.3
    data_structure:                grid
    title:                     

In [8]:
ds_day = ds_ca.sel(time="2025-03-18")
ds_day = ds_day.isel(time=3)
# Extract u and v wind components
u = ds_day['uwnd']  # zonal (east-west)
v = ds_day['vwnd']  # meridional (north-south)
uwnd_point = ds_day['uwnd'].sel(latitude=35, longitude=-120, method="nearest")
print(uwnd_point.values)


2.2123933


In [10]:
!pip install netCDF4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 73.1 MB/s eta 0:00:00


In [11]:
output_path = "/content/drive/MyDrive/UW_DSO_Project/Wind Data/Wind_data_2025.nc"

# Save compressed NetCDF
ds_ca.to_netcdf(
    output_path,
    mode="w",
    format="NETCDF4",
    engine="netcdf4",
    encoding={var: {"zlib": True, "complevel": 4} for var in ds_ca.data_vars}
)

print(f"Saved to {output_path}")


Saved to /content/drive/MyDrive/UW_DSO_Project/Wind Data/Wind_data_2025.nc
